In [1]:
# Libraries
import numpy as np
import pandas as pd
import json 
import jsonschema as js
import datetime
from pymongo import MongoClient


# update schema when connecting to database

In [ ]:
# connect to MongoDB client
client = MongoClient("mongodb://localhost:27017/")
db= client.Johan

# Compute allowed values
allowed_specimens = db.specimens.distinct("_id")
allowed_setups = db.exp_set_up.distinct("_id")

print(f"Allowed specimens: {len(allowed_specimens)}")
print(f"Allowed setups: {len(allowed_setups)}")

# Define JSON Schema validator dynamically
new_validator ={
    "$jsonSchema": {
      "bsonType": "object",
      "required": ['specimen_unique_id','set_up_unique_id'],
      "properties": {
        "specimen_unique_id": {
          "bsonType": "string",
          "enum": allowed_specimens, 
          "description": "must be one of the allowed specimen IDs"
        },
        "set_up_unique_id": {
          "bsonType": "string",
          "enum": allowed_setups,
          "description": "must be one of the allowed setup IDs"
        }
      }
    }
}
# 4️⃣ Update collection validator using collMod
result = db.command({
    "collMod": "measurements",
    "validator": new_validator,
    "validationLevel": "strict",
    "validationAction": "error"
})

print(" Validator updated successfully!")
print(result)

# populate specimen

In [ ]:
db.measurements.insert_one({
    "specimen_id": allowed_specimens[0],
    "setup_id": allowed_setups[0],
    "Data": data
})

# Auto-Updating Schema Validator

In [ ]:
from pymongo import MongoClient
from bson import ObjectId
import threading
import time

# 1️⃣ Connect to MongoDB
client = MongoClient("mongodb://localhost:27017/?replicaSet=rs0")  # must be replica set
db = client["lab_database"]

# --- Helper: rebuild and apply the validator
def update_validator():
    print("🔄 Updating validator...")

    allowed_specimens = db.specimens.distinct("_id")
    allowed_setups = db.exp_set_up.distinct("setup_id")

    new_validator = {
        "$jsonSchema": {
            "bsonType": "object",
            "required": ["specimen_id", "setup_id", "Data"],
            "properties": {
                "specimen_id": {
                    "bsonType": "objectId",
                    "enum": allowed_specimens,
                    "description": "Must reference a valid specimen _id"
                },
                "setup_id": {
                    "bsonType": "string",
                    "enum": allowed_setups,
                    "description": "Must reference a valid setup_id"
                },
                "Data": {
                    "bsonType": "array",
                    "minItems": 2048,
                    "maxItems": 2048,
                    "items": {
                        "bsonType": "array",
                        "minItems": 2,
                        "maxItems": 2,
                        "items": { "bsonType": "double" }
                    },
                    "description": "2048x2 array of doubles"
                }
            }
        }
    }

    db.command({
        "collMod": "measurements",
        "validator": new_validator,
        "validationLevel": "strict",
        "validationAction": "error"
    })
    print("✅ Validator updated successfully!\n")

# --- Initial validator build
update_validator()

# --- Helper: watch changes in collections
def watch_changes(collection_name):
    print(f"👀 Watching {collection_name} for changes...")
    with db[collection_name].watch() as stream:
        for change in stream:
            print(f"📦 Change detected in {collection_name}: {change['operationType']}")
            update_validator()

# --- Start watchers in background threads
threading.Thread(target=watch_changes, args=("specimens",), daemon=True).start()
threading.Thread(target=watch_changes, args=("exp_set_up",), daemon=True).start()

# Keep the main thread alive
while True:
    time.sleep(60)
